# Data Exploration using Pandas

#### The Panama Papers are 11.5 million leaked documents that detail financial and attorney–client information for more than 214,488 offshore entities. The documents, some dating back to the 1970s, were created by, and taken from, Panamanian law firm and corporate service provider Mossack Fonseca, and were leaked in 2015 by an anonymous source.

In [30]:
# load libraries
import pandas as pd
import numpy as np

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns # this isn't actually required, but it makes our plots look nice
import ipywidgets

#%matplotlib inline

import random

In [73]:
# Loading the data into pandas for easy processing
Address = pd.read_csv("address.csv", low_memory=False)
Entities = pd.read_csv("Entities.csv", low_memory=False)
Intermediaries = pd.read_csv("Intermediaries.csv", low_memory=False)
Officers = pd.read_csv("Officers.csv", low_memory=False)
Edges = pd.read_csv("Edges.csv", low_memory=False)

## Data Cleaning

In [89]:
df = pd.DataFrame(Officers['name'].value_counts().reset_index())
df.columns = ['name','count']
df.head(20)

,name,count
0,The Bearer,72098
1,EL PORTADOR,9325
2,THE BEARER,114
3,CHARITABLE AND GOODWILL FOUNDATION,92
4,NORTH ATLANTIC SERVICES LIMITED,91
5,FORMIA LIMITED,77
6,MAYO NOMINEES LIMITED,76
7,FIRST COURT LIMITED,76
8,MAYO SECRETARIES LIMITED,71
9,RICHMOND NOMINEES LIMITED,66


In [90]:
Officers['name'] = Officers['name'].replace('THE BEARER', 'The Bearer')


In [84]:
Officers['name'] = Officers['name'].replace(['TO THE BEARER'], 'The Bearer')


In [77]:
Officers['name'] = Officers['name'].replace(['THE BEARER', 'BEARER','Bearer','THE BEARER','the bearer','The Bearer'], 'The Bearer')


In [88]:
Officers.head(20)

,Unnamed: 0,node_id,name,country_codes,countries,sourceID,valid_until,note,node_type
0,0,12000001,KIM SOO IN,KOR,South Korea,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
1,1,12000002,Tian Yuan,CHN,China,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
2,2,12000003,GREGORY JOHN SOLOMON,AUS,Australia,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
3,3,12000004,MATSUDA MASUMI,JPN,Japan,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
4,4,12000005,HO THUY NGA,VNM,Viet Nam,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
5,5,12000006,RACHMAT ARIFIN,AUS,Australia,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
6,6,12000007,TAN SUN-HUA,PHL,Philippines,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
7,7,12000008,Ou Yang Yet-Sing and Chang Ko,TWN,Taiwan,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
8,8,12000009,Wu Chi-Ping and Wu Chou Tsan-Ting,TWN,Taiwan,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers
9,9,12000010,ZHONG LI MING,CHN,China,Panama Papers,The Panama Papers data is current through 2015,NaN,Officers


In [79]:
(Officers['name'].unique())

array(['KIM SOO IN', 'Tian Yuan', 'GREGORY JOHN SOLOMON', ...,
       'Arkady Rotenberg', 'Boris Rotenberg', 'Jean-Claude N’Da Ametchi'],
      dtype=object)

### Countries with entites who opened most offshore offices 

In [39]:
Entities

,Unnamed: 0,node_id,name,jurisdiction,jurisdiction_description,country_codes,countries,incorporation_date,inactivation_date,struck_off_date,closed_date,ibcRUC,status,company_type,service_provider,sourceID,valid_until,note,node_type
0,0,10000001,"TIANSHENG INDUSTRY AND TRADING CO., LTD.",SAM,Samoa,HKG,Hong Kong,23-MAR-2006,18-FEB-2013,15-FEB-2013,NaN,25221,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
1,1,10000002,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",SAM,Samoa,HKG,Hong Kong,27-MAR-2006,27-FEB-2014,15-FEB-2014,NaN,25249,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
2,2,10000003,"HOTFOCUS CO., LTD.",SAM,Samoa,HKG,Hong Kong,10-JAN-2006,15-FEB-2012,15-FEB-2012,NaN,24138,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
3,3,10000004,"SKY-BLUE GIFTS & TOYS CO., LTD.",SAM,Samoa,HKG,Hong Kong,06-JAN-2006,16-FEB-2009,15-FEB-2009,NaN,24012,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
4,4,10000005,FORTUNEMAKER INVESTMENTS CORPORATION,SAM,Samoa,HKG,Hong Kong,19-APR-2006,15-MAY-2009,15-FEB-2008,NaN,R25638,Changed agent,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213629,213629,10214482,MILLION TECH DEVELOPMENT LTD.,BVI,British Virgin Islands,HKG,Hong Kong,05-JAN-2010,01-NOV-2011,31-OCT-2011,NaN,NaN,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
213630,213630,10214483,FAITH CONNECTION INTERNATIONAL LTD.,BVI,British Virgin Islands,HKG,Hong Kong,05-JAN-2010,01-NOV-2011,31-OCT-2011,NaN,NaN,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
213631,213631,10214484,LARK OVERSEAS INVESTMENT LIMITED,BVI,British Virgin Islands,HKG,Hong Kong,04-JAN-2010,01-NOV-2013,31-OCT-2013,NaN,NaN,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
213632,213632,10214486,ASTRA SKIES LIMITED,BVI,British Virgin Islands,ARE,United Arab Emirates,15-DEC-2009,09-DEC-2011,30-APR-2012,NaN,NaN,Changed agent,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity


In [40]:
Entities['service_provider'].value_counts()

Mossack Fonseca    213634
Name: service_provider, dtype: int64

**The service provider for all the entities is Mossack Fonseca.However the role of the firm in the establishment of these entities from the dataframe alone is unclear.**

In [37]:
x_ent = pd.DataFrame(Entities['countries'].value_counts()).reset_index()
x_ent.columns = ['countries','count']
x_ent

,countries,count
0,Hong Kong,37911
1,Switzerland,37911
2,Panama,15811
3,Jersey,14331
4,Luxembourg,10840
...,...,...
154,Liberia,1
155,Yemen,1
156,Georgia,1
157,American Samoa,1


In [38]:
plt.style.use("fivethirtyeight")
@ipywidgets.interact(count=[1000,500,100,1],color=['olive','maroon','green','blue','brown','cyan','dimgrey','salmon','darkseagreen','royalblue','lightgreen'])
def plot_bars(count,color):
    #figure(figsize=(15,7))
    #figure(figsize=(15,7))
    filtered_df = x_ent[x_ent['count'] > count]
    filtered_df.plot(kind='bar', x='countries', y='count',color=color,figsize=[20,5])
    plt.title("Countries with entites who open offshore accounts",fontsize=15,fontweight='bold')
    plt.ylabel("Number of offices",fontsize=15)

    #plt.figure()
    plt.show()
#     plt.bar(x = np.array(x[x['count']>1000].countries), height = np.array(x[x['count']>1000].count), color =color,width = 0.8)
    #plt.show()

interactive(children=(Dropdown(description='count', options=(1000, 500, 100, 1), value=1000), Dropdown(descrip…

**Most of the corporate clients of Mossack Fonseca were from Hong Kong, Switzerland, the United Kingdom, Luxembourg, Panama, and Cyprus.**

In [95]:
Entities

,Unnamed: 0,node_id,name,jurisdiction,jurisdiction_description,country_codes,countries,incorporation_date,inactivation_date,struck_off_date,closed_date,ibcRUC,status,company_type,service_provider,sourceID,valid_until,note,node_type
0,0,10000001,"TIANSHENG INDUSTRY AND TRADING CO., LTD.",SAM,Samoa,HKG,Hong Kong,2006-03-23,18-FEB-2013,15-FEB-2013,NaN,25221,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
1,1,10000002,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",SAM,Samoa,HKG,Hong Kong,2006-03-27,27-FEB-2014,15-FEB-2014,NaN,25249,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
2,2,10000003,"HOTFOCUS CO., LTD.",SAM,Samoa,HKG,Hong Kong,2006-01-10,15-FEB-2012,15-FEB-2012,NaN,24138,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
3,3,10000004,"SKY-BLUE GIFTS & TOYS CO., LTD.",SAM,Samoa,HKG,Hong Kong,2006-01-06,16-FEB-2009,15-FEB-2009,NaN,24012,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
4,4,10000005,FORTUNEMAKER INVESTMENTS CORPORATION,SAM,Samoa,HKG,Hong Kong,2006-04-19,15-MAY-2009,15-FEB-2008,NaN,R25638,Changed agent,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213629,213629,10214482,MILLION TECH DEVELOPMENT LTD.,BVI,British Virgin Islands,HKG,Hong Kong,2010-01-05,01-NOV-2011,31-OCT-2011,NaN,NaN,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
213630,213630,10214483,FAITH CONNECTION INTERNATIONAL LTD.,BVI,British Virgin Islands,HKG,Hong Kong,2010-01-05,01-NOV-2011,31-OCT-2011,NaN,NaN,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
213631,213631,10214484,LARK OVERSEAS INVESTMENT LIMITED,BVI,British Virgin Islands,HKG,Hong Kong,2010-01-04,01-NOV-2013,31-OCT-2013,NaN,NaN,Defaulted,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity
213632,213632,10214486,ASTRA SKIES LIMITED,BVI,British Virgin Islands,ARE,United Arab Emirates,2009-12-15,09-DEC-2011,30-APR-2012,NaN,NaN,Changed agent,NaN,Mossack Fonseca,Panama Papers,The Panama Papers data is current through 2015,NaN,Entity


In [96]:
# Convert to datetime
Entities["incorporation_date"] = pd.to_datetime(Entities["incorporation_date"],format='%d-%b-%Y',errors='coerce')

In [97]:
Entities["incorporation_year"]=Entities["incorporation_date"].dt.year

In [102]:
#Effective Date
Entities["inactivation_date"] = pd.to_datetime(Entities["inactivation_date"],
                                           format='%d-%b-%Y',errors='coerce')

In [103]:
Entities["inactivation_year"] = Entities["inactivation_date"].dt.year

In [98]:
x_year = pd.DataFrame((Entities['incorporation_year'].value_counts()).reset_index())
x_year.columns = ['incorporation_year','count']
x_year

,incorporation_year,count
0,2005.0,13246
1,2007.0,12814
2,2006.0,12355
3,2004.0,11014
4,2008.0,10836
...,...,...
59,1936.0,1
60,1961.0,1
61,1945.0,1
62,1951.0,1


In [100]:
plt.style.use("fivethirtyeight")
@ipywidgets.interact(count=[5000,1000,500,100,1],color=['olive','maroon','green','blue','brown','cyan','dimgrey','salmon','darkseagreen','royalblue','lightgreen'])
def plot_bars(count,color):
    filtered_df_ent = x_year[x_year['count'] > count]
    filtered_df_ent.plot( x='incorporation_year', y='count',color=color,figsize=[20,5])   
    plt.title("Yearly Incorporation counts",fontsize=15,fontweight='bold')
    plt.ylabel("Number of offshore offices opened",fontsize=15)
    plt.show()


interactive(children=(Dropdown(description='count', options=(5000, 1000, 500, 100, 1), value=5000), Dropdown(d…

**Number of offshore entities established increased after 1990.**

In [104]:
x_year_inactivation = pd.DataFrame((Entities['inactivation_year'].value_counts()).reset_index())
x_year_inactivation.columns = ['inactivation_year','count']
x_year_inactivation

,inactivation_year,count
0,2010.0,9973
1,2009.0,9072
2,2014.0,9017
3,2013.0,8913
4,2015.0,8862
5,2011.0,8163
6,2008.0,7507
7,2006.0,7440
8,2012.0,7024
9,2007.0,6693


In [106]:
plt.style.use("fivethirtyeight")
@ipywidgets.interact(count=[5000,1000,500,100,1],color=['olive','maroon','green','blue','brown','cyan','dimgrey','salmon','darkseagreen','royalblue','lightgreen'])
def plot_bars(count,color):
    filtered_df_ent = x_year_inactivation[x_year_inactivation['count'] > count]
    filtered_df_ent.plot( x='inactivation_year', y='count',color=color,figsize=[20,5])   
    plt.title("Yearly Incorporation counts",fontsize=15,fontweight='bold')
    plt.ylabel("Number of offshore offices opened",fontsize=15)
    plt.show()


interactive(children=(Dropdown(description='count', options=(5000, 1000, 500, 100, 1), value=5000), Dropdown(d…

## Tax Havens

In [113]:
Address

,Unnamed: 0,node_id,name,address,country_codes,countries,sourceID,valid_until,note,node_type
0,0,14000001,NaN,-\t27 ROSEWOOD DRIVE #16-19 SINGAPORE 737920,SGP,Singapore,Panama Papers,The Panama Papers data is current through 2015,NaN,address
1,1,14000002,NaN,"""Almaly Village"" v.5, Almaty Kazakhstan",KAZ,Kazakhstan,Panama Papers,The Panama Papers data is current through 2015,NaN,address
2,2,14000003,NaN,"""Cantonia"" South Road St Georges Hill Weybridg...",GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN,address
3,3,14000004,NaN,"""CAY-OS"" NEW ROAD; ST.SAMPSON; GUERNSEY; CHANN...",GGY,Guernsey,Panama Papers,The Panama Papers data is current through 2015,NaN,address
4,4,14000005,NaN,"""Chirag"" Plot No 652; Mwamba Road; Kizingo; Mo...",KEN,Kenya,Panama Papers,The Panama Papers data is current through 2015,NaN,address
...,...,...,...,...,...,...,...,...,...,...
93449,93449,14109121,NaN,ZOLDOR HOLDINGS CORP. PANAMA REPUBLIC OF PANAM...,PAN,Panama,Panama Papers,The Panama Papers data is current through 2015,NaN,address
93450,93450,14109122,NaN,ZONE RESOURCES LIMITED 3705 GLOUCESTER TOWER; ...,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,NaN,address
93451,93451,14109129,NaN,ZURELDA TRADING S.A. AKARA BLDG.; 24 DE CASTRO...,VGB,British Virgin Islands,Panama Papers,The Panama Papers data is current through 2015,NaN,address
93452,93452,14109130,NaN,ZURICH TRADING LIMITED AKARA BLDG.; 24 DE CAST...,VGB,British Virgin Islands,Panama Papers,The Panama Papers data is current through 2015,NaN,address


In [114]:
x_Address = pd.DataFrame((Address['countries'].value_counts()).reset_index())
x_Address.columns = ['countries','count']
x_Address

,countries,count
0,China,20267
1,Hong Kong,9147
2,United Kingdom,3996
3,British Virgin Islands,3467
4,Russia,3346
...,...,...
197,Guinea-Bissau,1
198,Saint Martin (French part),1
199,San Marino,1
200,Grenada,1


In [115]:
plt.style.use("fivethirtyeight")
@ipywidgets.interact(count=[5000,1000,500,100,1],color=['olive','maroon','green','blue','brown','cyan','dimgrey','salmon','darkseagreen','royalblue','lightgreen'])
def plot_bars(count,color):
    filtered_df_ent = x_Address[x_Address['count'] > count]
    filtered_df_ent.plot(kind='bar', x='countries', y='count',color=color,figsize=[20,5])   
    plt.title("Countries by intermediaries",fontsize=15,fontweight='bold')
    plt.ylabel("Number of officers",fontsize=15)
    plt.show()


interactive(children=(Dropdown(description='count', options=(5000, 1000, 500, 100, 1), value=5000), Dropdown(d…

**Most popular tax havens among the clients of Mossack Fonseca.**

In [107]:
x_officers = pd.DataFrame((Officers['countries'].value_counts()).reset_index())
x_officers.columns = ['countries','count']
x_officers

,countries,count
0,China,24635
1,Hong Kong,13362
2,British Virgin Islands,11231
3,Jersey,6892
4,Panama,5069
...,...,...
196,Niger,1
197,Saint Martin (French part),1
198,Burkina Faso,1
199,San Marino,1


In [108]:
plt.style.use("fivethirtyeight")
@ipywidgets.interact(count=[5000,1000,500,100,1],color=['olive','maroon','green','blue','brown','cyan','dimgrey','salmon','darkseagreen','royalblue','lightgreen'])
def plot_bars(count,color):
    filtered_df_ent = x_officers[x_officers['count'] > count]
    filtered_df_ent.plot(kind='bar', x='countries', y='count',color=color,figsize=[20,5])   
    plt.title("Countries by officers",fontsize=15,fontweight='bold')
    plt.ylabel("Number of officers",fontsize=15)
    plt.show()


interactive(children=(Dropdown(description='count', options=(5000, 1000, 500, 100, 1), value=5000), Dropdown(d…

In [110]:
x_intermediary = pd.DataFrame((Intermediaries['countries'].value_counts()).reset_index())
x_intermediary.columns = ['countries','count']
x_intermediary

,countries,count
0,Hong Kong,2202
1,United Kingdom,1367
2,Switzerland,1218
3,United States,603
4,Panama,552
...,...,...
236,Puerto Rico,1
237,Mauritius;United Kingdom,1
238,Jersey;Cyprus,1
239,Panama;Dominican Republic,1


In [112]:
plt.style.use("fivethirtyeight")
@ipywidgets.interact(count=[1000,500,100,1],color=['olive','maroon','green','blue','brown','cyan','dimgrey','salmon','darkseagreen','royalblue','lightgreen'])
def plot_bars(count,color):
    filtered_df_ent = x_intermediary[x_intermediary['count'] > count]
    filtered_df_ent.plot(kind='bar', x='countries', y='count',color=color,figsize=[20,5])   
    plt.title("Countries by intermediaries",fontsize=15,fontweight='bold')
    plt.ylabel("Number of officers",fontsize=15)
    plt.show()

interactive(children=(Dropdown(description='count', options=(1000, 500, 100, 1), value=1000), Dropdown(descrip…

In [ ]:
# Popular 

In [109]:
plt.style.use("fivethirtyeight")
@ipywidgets.interact(count=[5000,1000,500,100,1],color=['olive','maroon','green','blue','brown','cyan','dimgrey','salmon','darkseagreen','royalblue','lightgreen'])
def plot_bars(count,color):
    filtered_df_ent = x_officers[x_officers['count'] > count]
    filtered_df_ent.plot(kind='bar', x='countries', y='count',color=color,figsize=[20,5])   
    plt.title("Countries by officers",fontsize=15,fontweight='bold')
    plt.ylabel("Number of officers",fontsize=15)
    plt.show()


interactive(children=(Dropdown(description='count', options=(5000, 1000, 500, 100, 1), value=5000), Dropdown(d…

In [113]:
Address

,Unnamed: 0,node_id,name,address,country_codes,countries,sourceID,valid_until,note,node_type
0,0,14000001,NaN,-\t27 ROSEWOOD DRIVE #16-19 SINGAPORE 737920,SGP,Singapore,Panama Papers,The Panama Papers data is current through 2015,NaN,address
1,1,14000002,NaN,"""Almaly Village"" v.5, Almaty Kazakhstan",KAZ,Kazakhstan,Panama Papers,The Panama Papers data is current through 2015,NaN,address
2,2,14000003,NaN,"""Cantonia"" South Road St Georges Hill Weybridg...",GBR,United Kingdom,Panama Papers,The Panama Papers data is current through 2015,NaN,address
3,3,14000004,NaN,"""CAY-OS"" NEW ROAD; ST.SAMPSON; GUERNSEY; CHANN...",GGY,Guernsey,Panama Papers,The Panama Papers data is current through 2015,NaN,address
4,4,14000005,NaN,"""Chirag"" Plot No 652; Mwamba Road; Kizingo; Mo...",KEN,Kenya,Panama Papers,The Panama Papers data is current through 2015,NaN,address
...,...,...,...,...,...,...,...,...,...,...
93449,93449,14109121,NaN,ZOLDOR HOLDINGS CORP. PANAMA REPUBLIC OF PANAM...,PAN,Panama,Panama Papers,The Panama Papers data is current through 2015,NaN,address
93450,93450,14109122,NaN,ZONE RESOURCES LIMITED 3705 GLOUCESTER TOWER; ...,HKG,Hong Kong,Panama Papers,The Panama Papers data is current through 2015,NaN,address
93451,93451,14109129,NaN,ZURELDA TRADING S.A. AKARA BLDG.; 24 DE CASTRO...,VGB,British Virgin Islands,Panama Papers,The Panama Papers data is current through 2015,NaN,address
93452,93452,14109130,NaN,ZURICH TRADING LIMITED AKARA BLDG.; 24 DE CAST...,VGB,British Virgin Islands,Panama Papers,The Panama Papers data is current through 2015,NaN,address


In [114]:
x_Address = pd.DataFrame((Address['countries'].value_counts()).reset_index())
x_Address.columns = ['countries','count']
x_Address

,countries,count
0,China,20267
1,Hong Kong,9147
2,United Kingdom,3996
3,British Virgin Islands,3467
4,Russia,3346
...,...,...
197,Guinea-Bissau,1
198,Saint Martin (French part),1
199,San Marino,1
200,Grenada,1


In [115]:
plt.style.use("fivethirtyeight")
@ipywidgets.interact(count=[5000,1000,500,100,1],color=['olive','maroon','green','blue','brown','cyan','dimgrey','salmon','darkseagreen','royalblue','lightgreen'])
def plot_bars(count,color):
    filtered_df_ent = x_Address[x_Address['count'] > count]
    filtered_df_ent.plot(kind='bar', x='countries', y='count',color=color,figsize=[20,5])   
    plt.title("Countries by intermediaries",fontsize=15,fontweight='bold')
    plt.ylabel("Number of officers",fontsize=15)
    plt.show()


interactive(children=(Dropdown(description='count', options=(5000, 1000, 500, 100, 1), value=5000), Dropdown(d…

**Most popular tax havens among the clients of Mossack Fonseca.**